In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import time
import matplotlib.pyplot as plt

## Clustering as Numpy Arrays

In [ ]:
def apply_threshold_hsv(roi, hsv_color, threshold_h=10, threshold_s=35, threshold_v=80):
    """Apply color thresholding in HSV color space to isolate specific color ranges in the ROI."""
    
    lower_bound = np.array([max(0, hsv_color[0] - threshold_h), max(0, hsv_color[1] - threshold_s), max(0, hsv_color[2] - threshold_v)])
    upper_bound = np.array([min(180, hsv_color[0] + threshold_h), min(255, hsv_color[1] + threshold_s), min(255, hsv_color[2] + threshold_v)])
    mask = cv2.inRange(roi, lower_bound, upper_bound)
    return mask


def find_clusters(ref_img_hsv, hsv_color=(174, 131, 201), eps=10, min_size_threshold=90, max_size_threshold=300):
    """
    Find clusters of points in an image based on a given HSV color.

    Parameters:
    - ref_img_hsv (numpy.ndarray): The input image in HSV color space.
    - hsv_color (tuple): The target HSV color to find clusters around. Default is (174, 131, 201).
    - min_size_threshold (int): The minimum size threshold for a cluster to be considered. Default is 10.
    - max_size_threshold (int): The maximum size threshold for a cluster to be considered. Default is 300.

    Returns:
    - numpy.ndarray: An array of filtered clusters, where each cluster is represented as an array of points.

    Note:
    - The input image should be in HSV color space.
    - The returned clusters are filtered based on their size, where the size is determined by the number of points in the cluster.
    """
    
    mask = apply_threshold_hsv(ref_img_hsv, hsv_color)
    
    y_coord, x_coord = np.where(mask != 0)
    if len(y_coord) == 0:
        return np.array([])  # Return an empty numpy array if no points found
    
    coord_array = np.stack((x_coord, y_coord), axis=-1)
    dbscan = DBSCAN(eps=eps, min_samples=min_size_threshold)
    clusters = dbscan.fit_predict(coord_array)

    # Filter clusters based on size
    filtered_cluster_list = []
    for cluster_idx in np.unique(clusters):
        if cluster_idx != -1:
            cluster_points = coord_array[clusters == cluster_idx]
            if min_size_threshold <= len(cluster_points) <= max_size_threshold:
                filtered_cluster_list.append(cluster_points)
    return np.array(filtered_cluster_list, dtype=object)


def arrange_clusters(filtered_cluster_list):
    """
    Arrange clusters from left to right based on their mean x-coordinate.
    
    Parameters:
    - filtered_cluster_list: numpy.ndarray
        An array of filtered clusters, where each cluster is represented by an array of points.
        
    Returns:
    - numpy.ndarray
        An array of clusters sorted from left to right.
    """
    if len(filtered_cluster_list) == 0:
        return filtered_cluster_list  # Return the empty array if no clusters

    # Calculate the mean x-coordinate for each cluster
    mean_x_coords = [np.mean(cluster[:, 0]) for cluster in filtered_cluster_list]
    
    # Sort the clusters based on their mean x-coordinate
    sorted_indices = np.argsort(mean_x_coords)
    
    # Reorder the clusters based on the sorted indices
    sorted_clusters = filtered_cluster_list[sorted_indices]

    return sorted_clusters

## Centroid Calculation & Visualization

In [ ]:
def calculate_centroids(filtered_cluster_list):
    """
    Calculate the centroids of filtered clusters.

    Parameters:
    - filtered_cluster_list: numpy.ndarray
        An array of filtered clusters, where each cluster is represented by an array of points.
        
    Returns:
    - list
        A list of centroids of the clusters.
    """
    centroids = []
    for cluster in filtered_cluster_list:
        centroid_x = np.mean(cluster[:, 0])
        centroid_y = np.mean(cluster[:, 1])
        centroids.append((centroid_x, centroid_y))
    return centroids

## Calculating Displacements

In [ ]:
def calculate_displacements(ref_centroids, curr_centroids):
    """
    Calculate displacements of centroids from the reference frame to the current frame.
    
    Parameters:
    - ref_centroids: list of tuples
        The reference centroids from the initial frame.
    - curr_centroids: list of tuples
        The centroids from the current frame.
        
    Returns:
    - numpy.ndarray
        The displacements of centroids.
    """
    # Ensure that both centroid lists have the same number of centroids
    if len(ref_centroids) != len(curr_centroids):
        raise ValueError("The number of centroids in both frames does not match.")

    # Calculate displacements as differences in x and y coordinates
    displacements = np.array(curr_centroids) - np.array(ref_centroids)
    return displacements


## Isolating y-displacements

In [ ]:
# cap =  cv2.VideoCapture('output.mp4')
# cap =  cv2.VideoCapture('output_no_press.mp4')
# color = (174, 131, 201)

cap =  cv2.VideoCapture('cope.mp4')
color = (160,117,189)

# cap =  cv2.VideoCapture(0, cv2.CAP_DSHOW)     # UNCOMMENT THIS LINE FOR WINDOWS
cap =  cv2.VideoCapture(1)                     # UNCOMMENT THIS LINE FOR LINUX


ref_img_hsv = None
ref_centroids = None
count = 0
CAPTURE_COUNT = 20
eps = 6
min_size_threshold = 70
max_size_threshold = 200
displacement_threshold = 0.6        # CHANGE THIS FOR THE DISPLACEMENT THRESHOLD


while cap.isOpened():
    
    success, frame_img = cap.read()
    # frame_img = cv2.rotate(frame_img, cv2.ROTATE_180)

    
    if not success:
        print("Ignoring empty camera frame.")
        break

    # frame_img = cv2.medianBlur(frame_img, 3)
    if count < CAPTURE_COUNT:
        count += 1
        continue
    
    if ref_img_hsv is None:    
        ref_img_hsv = cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV)
        ref_clusters = find_clusters(ref_img_hsv, \
                                 hsv_color=color, \
                                 eps=eps, \
                                 min_size_threshold=min_size_threshold, \
                                 max_size_threshold=max_size_threshold)
        ref_clusters = arrange_clusters(ref_clusters)
        ref_centroids = calculate_centroids(ref_clusters)
        print("Captured Reference Image")
    
    if ref_img_hsv is not None:
        inference_img_hsv = cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV)
        inference_clusters = find_clusters(inference_img_hsv, \
                                           hsv_color=color, \
                                           eps=eps, \
                                           min_size_threshold=min_size_threshold, \
                                           max_size_threshold=max_size_threshold)
        inference_clusters = arrange_clusters(inference_clusters)
        inference_centroids = calculate_centroids(inference_clusters)
        
        # Drawing original (reference) centroids in blue
        for centroid in ref_centroids:
            cv2.circle(frame_img, (int(centroid[0]), int(centroid[1])), 3, (255, 0, 0), -1)\
        
        if len(inference_centroids) == len(ref_centroids):
            displacements = calculate_displacements(ref_centroids, inference_centroids)
            mean_x_dis = np.mean(displacements[:, 0])
            mean_y_dis = np.mean(displacements[:, 1])
            normalized = []
            for ix, iy in inference_centroids:
                normalized.append((ix-mean_x_dis , iy-mean_y_dis))
            
            displacements[:, 0] -= mean_x_dis
            displacements[:, 1] -= mean_y_dis
            
            key_press_indices = []
            for idx, dy in enumerate(displacements[:,1]):
                if dy > displacement_threshold:
                    key_press_indices.append(idx)
            
            print('Key presses:', key_press_indices)
            
            for idx, centroid in enumerate(normalized):
                if idx in key_press_indices:
                    # Drawing pressed centroids in RED
                    cv2.circle(frame_img, (int(centroid[0]), int(centroid[1])), 3, (0, 0, 255), -1)
                else:
                    # Drawing remaining centroids in GREEN
                    cv2.circle(frame_img, (int(centroid[0]), int(centroid[1])), 3, (0, 255, 0), -1)
        else:
            print("No. of centroids mismatch")
            
    # time.sleep(0.01)

    
    cv2.imshow('Pressed Key Frame', frame_img)
    
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break
    
cap.release()
cv2.destroyAllWindows()
